# Run and Deploy GPT2 Model for Text Generation with XLA Generate

#### Olayinka Peter


Goal here is to run a base gpt2 model and deploy it with tensorflow serving.





<br>

---



---


In [1]:
# Install dependencies
!pip install accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 111.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 95.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


In [2]:
# Note: execution times are deeply dependent on hardware.
import tensorflow as tf

import os
from transformers import (
    TFAutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    logging,
)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [3]:
# GPT Model from Huggingface
model_name = "gpt2"

In [5]:
# Load base model
model = TFAutoModelForCausalLM.from_pretrained(
    model_name
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# remember: decoder-only models need left-padding
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left", pad_token="</s>")
model = TFAutoModelForCausalLM.from_pretrained(model_name)

# tokenization and generation arguments
tokenization_kwargs = {"pad_to_multiple_of": 200, "padding": True, "return_tensors": "tf"}
generation_kwargs = {"num_beams": 4, "max_new_tokens": 200}

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [19]:
# Enable XLA for TensorFlow
tf.config.optimizer.set_jit(True)
tf.config.optimizer.set_experimental_options({"xla_gpu": True})

# Initialize pipeline with XLA-enabled TensorFlow
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=200,
    framework="tf",  # Set the framework to TensorFlow
)

# Run text generation pipeline with our next model
prompt = "The best thing about Japan is?"
result = pipe(f"{prompt}", **generation_kwargs)
print(result[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=32) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


The best thing about Japan is? It's a beautiful country.

The best thing about Japan is? It's a beautiful country.

The best thing about Japan is? It's


In [9]:
# Save the model

import time
t = time.time()

export_path_sm = "./{}".format(int(t))
print(export_path_sm)

tf.saved_model.save(model, export_path_sm)

./1692547477


In [10]:
# We can save the tokenizr as a pickle file

import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Reloading saved models

TensorFlow Serving allows to reload saved models without having to restart the server. This can be useful for updating models with new data or for debugging models.

In [12]:
# Load saved model
DEFAULT_FUNCTION_KEY = "serving_default"
loaded_model = tf.saved_model.load(export_path_sm)
inference_func = loaded_model.signatures[DEFAULT_FUNCTION_KEY]
print(loaded_model)
print(inference_func)

# Load saved tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
print(tokenizer)

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x78f97e723c70>
ConcreteFunction signature_wrapper(*, input_ids, attention_mask, token_type_ids)
  Args:
    attention_mask: int32 Tensor, shape=(None, None)
    input_ids: int32 Tensor, shape=(None, None)
    token_type_ids: int32 Tensor, shape=(None, None)
  Returns:
    {'logits': <1>, 'past_key_values_1': <2>, 'past_key_values_10': <3>, 'past_key_values_11': <4>, 'past_key_values_12': <5>, 'past_key_values_2': <6>, 'past_key_values_3': <7>, 'past_key_values_4': <8>, 'past_key_values_5': <9>, 'past_key_values_6': <10>, 'past_key_values_7': <11>, 'past_key_values_8': <12>, 'past_key_values_9': <13>}
      <1>: float32 Tensor, shape=(None, None, 50257)
      <2>: float32 Tensor, shape=(2, None, 12, None, 64)
      <3>: float32 Tensor, shape=(2, None, 12, None, 64)
      <4>: float32 Tensor, shape=(2, None, 12, None, 64)
      <5>: float32 Tensor, shape=(2, None, 12, None, 64)
      <6>

---

---

---
